In [ ]:
%pip install pennylane --upgrade

In [ ]:
%pip install pennylane-qiskit

In [ ]:
# %pip freeze | grep pennylane
%pip freeze > requirements.txt

## Quantum Circuits

In [ ]:
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def my_quantum_function(x, y):
    qml.RZ(x, wires=0)
    qml.CNOT(wires=[0,1])
    qml.RY(y, wires=1)
    return qml.expval(qml.PauliZ(1))

In [ ]:
shots_list = [5, 10, 1000]
dev = qml.device("default.qubit", wires=2, shots=shots_list)

In [ ]:
circuit = qml.QNode(my_quantum_function, dev)
circuit(np.pi/2, np.pi/2)
print(qml.draw(circuit)(np.pi/2, np.pi/2))

In [ ]:
qml.drawer.use_style("black_white")
fig, ax = qml.draw_mpl(circuit)(np.pi/2, np.pi/2)
plt.show()

In [ ]:
## QNode Decorator ##

dev = qml.device('default.qubit', wires=2, shots=10)

@qml.qnode(dev)
def circuit(x):
    qml.RZ(x, wires=0)
    qml.CNOT(wires=[0,1])
    qml.RY(x, wires=1)
    return qml.expval(qml.PauliZ(1))

result = circuit(np.pi/2)
result

## Loading and calling a parametrized Qiskit QuantumCircuit object while using the QNode decorator

In [26]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
import numpy as np

dev = qml.device('default.qubit', wires=2, shots=10)

theta = Parameter('θ')

qc = QuantumCircuit(2)
qc.rz(theta, [0])
qc.rx(theta, [0])
qc.cx(0, 1)

@qml.qnode(dev)
def quantum_circuit_with_loaded_subcircuit(x):
    qml.from_qiskit(qc)({theta: x})
    return qml.expval(qml.PauliZ(0))

angle = np.pi/2
result = quantum_circuit_with_loaded_subcircuit(angle)
result

np.float64(-0.6)